# **3188. Find Top Scoring Students II**
``` console
Table: students

+-------------+----------+
| Column Name | Type     | 
+-------------+----------+
| student_id  | int      |
| name        | varchar  |
| major       | varchar  |
+-------------+----------+
student_id is the primary key for this table. 
Each row contains the student ID, student name, and their major.

Table: courses

+-------------+-------------------+
| Column Name | Type              |       
+-------------+-------------------+
| course_id   | int               |    
| name        | varchar           |      
| credits     | int               |           
| major       | varchar           |       
| mandatory   | enum              |      
+-------------+-------------------+
course_id is the primary key for this table. 
mandatory is an enum type of ('Yes', 'No').
Each row contains the course ID, course name, credits, major it belongs to, and whether the course is mandatory.

Table: enrollments

+-------------+----------+
| Column Name | Type     | 
+-------------+----------+
| student_id  | int      |
| course_id   | int      |
| semester    | varchar  |
| grade       | varchar  |
| GPA         | decimal  | 
+-------------+----------+
(student_id, course_id, semester) is the primary key (combination of columns with unique values) for this table.
Each row contains the student ID, course ID, semester, and grade received.
Write a solution to find the students who meet the following criteria:

Have taken all mandatory courses and at least two elective courses offered in their major.
Achieved a grade of A in all mandatory courses and at least B in elective courses.
Maintained an average GPA of at least 2.5 across all their courses (including those outside their major).
Return the result table ordered by student_id in ascending order.

 

Example:

Input:

students table:

 +------------+------------------+------------------+
 | student_id | name             | major            |
 +------------+------------------+------------------+
 | 1          | Alice            | Computer Science |
 | 2          | Bob              | Computer Science |
 | 3          | Charlie          | Mathematics      |
 | 4          | David            | Mathematics      |
 +------------+------------------+------------------+
 
courses table:

 +-----------+-------------------+---------+------------------+----------+
 | course_id | name              | credits | major            | mandatory|
 +-----------+-------------------+---------+------------------+----------+
 | 101       | Algorithms        | 3       | Computer Science | yes      |
 | 102       | Data Structures   | 3       | Computer Science | yes      |
 | 103       | Calculus          | 4       | Mathematics      | yes      |
 | 104       | Linear Algebra    | 4       | Mathematics      | yes      |
 | 105       | Machine Learning  | 3       | Computer Science | no       |
 | 106       | Probability       | 3       | Mathematics      | no       |
 | 107       | Operating Systems | 3       | Computer Science | no       |
 | 108       | Statistics        | 3       | Mathematics      | no       |
 +-----------+-------------------+---------+------------------+----------+
 
enrollments table:

 +------------+-----------+-------------+-------+-----+
 | student_id | course_id | semester    | grade | GPA |
 +------------+-----------+-------------+-------+-----+
 | 1          | 101       | Fall 2023   | A     | 4.0 |
 | 1          | 102       | Spring 2023 | A     | 4.0 |
 | 1          | 105       | Spring 2023 | A     | 4.0 |
 | 1          | 107       | Fall 2023   | B     | 3.5 |
 | 2          | 101       | Fall 2023   | A     | 4.0 |
 | 2          | 102       | Spring 2023 | B     | 3.0 |
 | 3          | 103       | Fall 2023   | A     | 4.0 |
 | 3          | 104       | Spring 2023 | A     | 4.0 |
 | 3          | 106       | Spring 2023 | A     | 4.0 |
 | 3          | 108       | Fall 2023   | B     | 3.5 |
 | 4          | 103       | Fall 2023   | B     | 3.0 |
 | 4          | 104       | Spring 2023 | B     | 3.0 |
 +------------+-----------+-------------+-------+-----+
 
Output:

 +------------+
 | student_id |
 +------------+
 | 1          |
 | 3          |
 +------------+
 
Explanation:

Alice (student_id 1) is a Computer Science major and has taken both Algorithms and Data Structures, receiving an A in both. She has also taken Machine Learning and Operating Systems as electives, receiving an A and B respectively.

Bob (student_id 2) is a Computer Science major but did not receive an A in all required courses.

Charlie (student_id 3) is a Mathematics major and has taken both Calculus and Linear Algebra, receiving an A in both. He has also taken Probability and Statistics as electives, receiving an A and B respectively.

David (student_id 4) is a Mathematics major but did not receive an A in all required courses.

Note: Output table is ordered by student_id in ascending order.

```

**Solution-1:**

``` sql
# Write your MySQL query statement below

select distinct student_id from
(select e.student_id, e.course_id, e.grade, e.gpa, c.major, c.mandatory,
SUM(mandatory = 'yes' and grade = 'A' and s.major = c.major) OVER(partition by e.student_id,s.major) as std_mndtry_cnt,
crs_mndtry_cnt,
SUM(mandatory = 'no' and e.grade in ('A','B')) OVER(partition by e.student_id,s.major) as std_non_mndtry_cnt,
AVG(e.gpa)OVER(partition by e.student_id) as average_gpa,
crs_non_mndtry_cnt
FROM enrollments e
INNER JOIN (
select course_id, major, mandatory,
SUM(mandatory = 'yes') OVER(partition by major) as crs_mndtry_cnt,
SUM(mandatory = 'no') OVER(partition by major) as crs_non_mndtry_cnt
FROM courses )c
ON e.course_id = c.course_id
INNER JOIN students s
ON e.student_id = s.student_id AND s.major = c.major) a
WHERE (std_mndtry_cnt = crs_mndtry_cnt and std_non_mndtry_cnt >= 2) and round(average_gpa,1) >= 2.5
ORDER BY student_id 

``` sql
# Write your MySQL query statement below
WITH MandatoryCourses AS (
    SELECT c.course_id, c.major
    FROM courses c
    WHERE c.mandatory = 'Yes'
),
ElectiveCourses AS (
    SELECT c.course_id, c.major
    FROM courses c
    WHERE c.mandatory = 'No'
),
StudentMandatory AS (
    SELECT e.student_id, s.major
    FROM enrollments e
    JOIN MandatoryCourses m ON e.course_id = m.course_id
    JOIN students s ON e.student_id = s.student_id AND s.major = m.major
    WHERE e.grade = 'A'
    GROUP BY e.student_id, s.major
    HAVING COUNT(DISTINCT e.course_id) = (SELECT COUNT(*) FROM MandatoryCourses WHERE major = s.major)
),
StudentElective AS (
    SELECT e.student_id, s.major
    FROM enrollments e
    JOIN ElectiveCourses ec ON e.course_id = ec.course_id
    JOIN students s ON e.student_id = s.student_id AND s.major = ec.major
    WHERE e.grade IN ('A', 'B')
    GROUP BY e.student_id, s.major
    HAVING COUNT(DISTINCT e.course_id) >= 2
),
AverageGPA AS (
    SELECT e.student_id, AVG(e.GPA) AS avg_gpa
    FROM enrollments e
    GROUP BY e.student_id
    HAVING AVG(e.GPA) >= 2.5
)
SELECT DISTINCT sm.student_id
FROM StudentMandatory sm
JOIN StudentElective se ON sm.student_id = se.student_id
JOIN AverageGPA ag ON sm.student_id = ag.student_id
ORDER BY sm.student_id;

`